# Расчёт погрешностей измерений и правила их построения

## Вычисление погрешностей данных

_Упражнение 11.1_

Создайте функцию расчёта средней квадратичной погрешности, напомню формулу:
$$s=\sqrt{\frac{1}{N(N-1)}\sum_{k=1}^N(X_k-X_{вр})^2}$$

In [ ]:
import numpy as np

def std(Y, Y_mean):
    N = len(Y)
    return np.sqrt(np.sum(np.power(Y-Y_mean, 2))/(N-1)/N)


В качестве наиболее вероятного значения можно взять среднее при одинаковых параметрах или полученное фиттированием, например, функцией `curve_fit`. 

Доработайте вашу функцию таким расчётом:<br>
    <p style="text-indent: 40px">В одном случаи считается среднее на основе переданного массива;</p>
    <p style="text-indent: 40px">В другом принимается в его качестве - учтите разные типы данных</p>

In [ ]:
def std_mean(Y: list | tuple | np.ndarray, 
             calc_mean: str | float | int | np.floating | np.integer='Average') -> np.floating:
    """
    Расчёт средней квадратичной погрешности
    :param Y: Принимает два значения:
        str - вероятное значение считается как среднее арифметическое из всех переданных в функцию значений массива
        int | float | np.float64 | ... - эталонное значение 
    """
    N = len(Y)
    Y_mean = np.mean(Y) if type(calc_mean) is str else calc_mean
    return np.sqrt(np.sum(np.power(Y-Y_mean, 2))/(N-1)/N)

_Упражнение 11.2_

Усложним задачу: сейчас наша функция считает только одну точку, теперь будем применять вычисление сразу для нескольких. 

На входе имеем массив, содержащий набор значений, который требуется разбить на блоки. Один блок даст одну среднюю точку и данные для расчёта отклонения. В первую очередь, требуется разбить весь массив на такие блоки.

Сделаем два способа задания массивов для отображения:
1) Аргументом передаётся число, задающее третий параметр среза по всему массиву `X`
2) Кортеж или список из 3-х элементов среза, по которому будут вычислены погрешности

_*Удобнее отработать данный выбор отдельно от прочего функционала, а уже затем встраивать в функцию_

In [ ]:
def std_massive(X, x_points: int | tuple | np.ndarray | list = 1) -> np.floating:
    if type(x_points) is int:
        x_points = X[::x_points]
    elif type(x_points) is tuple and len(x_points) == 3:
        x_points = X[x_points[0]:x_points[1]:x_points[2]]

Теперь есть массив координат, в которых высчитываются наиболее вероятные значения. Добавим её в результат _упражнения 1.1_ для случая расчёта среднего арифметического(следующим шагом будет реализация `curve_fit`). Таким образом, **x_points** будет содержать элементы массива `X`, по которым будет происходить разбиение на группы усреднения:

~~~python
X = [11, 12, 13, 21, 22, 23]
x_point = (0, len(X), 3) or 3
~~~

Допишите функцию так, чтобы на вход подавались данные в виде одного массива вместе с индексами разбиения на блоки, а на выходе массив рассчитанных погрешностей, соответствующих порядку блоков

In [ ]:
def std_massive(X, Y, x_points: int | tuple | np.ndarray | list = 1) -> list[np.floating]:
    if type(x_points) is int:
        N = x_points
        indexes = np.arange(0, len(X)+1, x_points)
        x_points = X[::x_points]
    elif type(x_points) is tuple and len(x_points) == 3:
        N = x_points[2]
        indexes = np.arange(x_points[0], x_points[1]+1, x_points[2])
        x_points = X[x_points[0]:x_points[1]:x_points[2]]
    result = []
    for i, index in enumerate(indexes[1:]):
        Y_cut = Y[int(indexes[i]):int(index)]
        Y_mean = np.mean(Y_cut)
        result.append(np.sqrt(np.sum(np.power(Y_cut-Y_mean, 2))/(N-1)/N))
    return result

std_massive([11, 12, 13, 21, 22, 23], [11, 12, 13, 21, 22, 23], x_points=(0, 6, 3))

_Упражнение 11.3_

Попробуем написать новую функцию с расчётом через фиттирование. Эта часть будет состоять из двух частей:
1) Применение `curve_fit` для получения эталонного значения
2) Расчёт отклонений в выбранных точках по соседним

Для первого пункта потребуется аргумент с функцией, по которой будем искать общий вид. 

Предлагаем ввести аргумент, задающий диапазон поиска значений для расчёта ошибок через отличие от эталонного _x_, координаты которых передаются массивом. То есть, блок для расчёта погрешности будет задаваться точками, координаты _x_ которых укладываются в диапозон $[x-interval, x+interval]$

Эталонное значение получаем из результата работы `curve_fit` в точках переданного массива для построения.

Аргументы функции такие: 
1) `X`, `Y` - массивы данных
2) `x_points` - координаты точек для расчёта погрешностей
3) `interval` - задаёт диапазон для получения блока значений

На выходе всё тот же массив со значениями отклонений и параметрами от `curve_fit`

In [ ]:
# На подумать

Теперь имеем готовый код для получения массивов случайных ошибок с двумя методами расчёта

**Задача 4.3** Возьмите одну из кривых с полусеместрового контроля и постройте для неё средне квадратичные значения погрешностей с учётом правил оформления. (Учтите наличие двух значений `Y` при одном `X`)

**Задача 4.4** Повторите оформление и обработку данных из файла `data_43.txt` из статьи

<img src="Graph_from_article.jpg" width="700" height="600" />